# Python для анализа данных

*Ян Пиле, НИУ ВШЭ*

## Интерактивные визуализации в Plotly v2

In [1]:
!pip install cufflinks
!pip install fastparquet
!pip install pyarrow

  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=68736 sha256=973d18990a03f04fb91ae62d6484feade9dfe02662be1bb9b87965abd8024573
  Stored in directory: c:\users\mbbur\appdata\local\pip\cache\wheels\29\b4\f8\2fd2206eeeba6ccad8167e4e8894b8c4ec27bf1342037fd136
Successfully built cufflinks
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


In [2]:
# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [3]:
import pandas as pd
df = pd.read_parquet('medium_data_2019_01_06')
df.head()

,claps,days_since_publication,fans,link,num_responses,publication,published_date,read_ratio,read_time,reads,...,type,views,word_count,claps_per_word,editing_days,<tag>Education,<tag>Data Science,<tag>Towards Data Science,<tag>Machine Learning,<tag>Python
119,2,574.858594,2,https://medium.com/p/screw-the-environment-but...,0,None,2017-06-10 14:25:00,41.98,7,68,...,published,162,1859,0.001076,0,0,0,0,0,0
118,18,567.540639,3,https://medium.com/p/the-vanquishing-of-war-pl...,0,None,2017-06-17 22:02:00,32.93,14,54,...,published,164,3891,0.004626,0,0,0,0,0,0
121,50,554.920762,19,https://medium.com/p/capstone-project-mercedes...,0,None,2017-06-30 12:55:00,20.19,42,215,...,published,1065,12025,0.004158,0,0,0,0,1,1
122,0,554.078160,0,https://medium.com/p/home-of-the-scared-5af0fe...,0,None,2017-07-01 09:08:00,35.85,9,19,...,published,53,2533,0.000000,0,0,0,0,0,0
114,0,550.090507,0,https://medium.com/p/the-triumph-of-peace-f485...,0,None,2017-07-05 08:51:00,8.77,14,5,...,published,57,3892,0.000000,1,0,0,0,0,0


Типы графиков, которые можно нарисовать с помощью cufflinks:

   
    scatter, bar, box, spread
    ratio, heatmap, surface
    histogram, bubble, bubble3d
    scatter3d, scattergeo, ohlc
    candle, pie, choropleth

**Распределение одной переменной**

Здесь очевидным решением будет гистограмма

In [4]:
df['claps'].iplot(
    kind='hist',
    bins=30,
    xTitle='claps',
    linecolor='black',
    yTitle='count',
    title='Claps Distribution')

На любой прямоугольник можно навести курсор и увидеть значение

**Гистограмма с процентами**

Точно такая же гистограмма в одно действие превращается в распределение с процентами.

In [5]:
df['reads'].iplot(
    kind='hist',
    bins=30,
    xTitle='reads',
    linecolor='black',
    histnorm='percent',
    yTitle='percentage (%)',
    title='Reads Distribution in Percent')

Также на одну гистограмму можно уложить несколько распределений (впрочем, мы это уже проделывали)

In [6]:
def to_time(dt):
    return dt.hour + dt.minute / 60

In [7]:
df['time_started'] = df['started_date'].apply(to_time)
df['time_published'] = df['published_date'].apply(to_time)

df[['time_started', 'time_published']].iplot(
    kind='hist',
    linecolor='black',
    bins=24,
    histnorm='percent',
    bargap=0.1,
    opacity=0.8,
    xTitle='Time of Day',
    yTitle='(%) of Articles',
    title='Time Started and Time Published')

А иногда хочется, чтобы распределении не строились рядом друг с другом, а строились одно поверх другого (с перекрытием). За это отвечает аргумент *barmode* которому надо присвоить значение overlay.

In [8]:
df[['time_published', 'time_started']].iplot(
    kind='hist',
    bins=24,
    linecolor='black',
    opacity=0.6,
    histnorm='percent',
    barmode='overlay',
    xTitle='Time of day',
    yTitle='(%) of articles',
    title='Time Started and Time Published Overlaid')

Столбчатые диаграммы обычно строятся поверх группировки датафрейма по некоторому полю (обычно категориальному). Например давайте посчитаем количество статей в публикации.

In [9]:
df['publication'].unique()

array(['None', 'Towards Data Science', 'Engineering @ Feature Labs',
       'Noteworthy - The Journal Blog', 'The Reality Project'],
      dtype=object)

In [10]:
df.groupby('publication').count()['fans'].iplot(
    kind='bar', yTitle='Number of Articles', linecolor='black', title='Articles by Publication')

А еще можно строить столбчатые диаграммы по нескольким столбцам (Если в них однородная информация - у нас, например, есть флажки того, что публикация содержит некий тег (например, Python))

In [50]:
[c for c in df if '<tag>' in c]

['<tag>Education',
 '<tag>Data Science',
 '<tag>Towards Data Science',
 '<tag>Machine Learning',
 '<tag>Python']

In [11]:
df[[c for c in df if '<tag>' in c]].sum().iplot(
    kind='bar',
    xTitle='Tag',
    yTitle='Number of Articles with Tag',
    title='Frequency of Tags',
    linecolor='black',
    sortbars=True)

**А еще можно распределение пары признаков рядом нарисовать**

Установим в качестве индекса published_date, приведем дату к месяцу и усредним :).

In [12]:
df2 = df[['views', 'reads',
          'published_date']].set_index('published_date').resample('M').mean()
df2.head()

,views,reads
published_date,,
2017-06-30,463.666667,112.333333
2017-07-31,5521.333333,1207.166667
2017-08-31,6242.800000,993.700000
2017-09-30,2113.000000,279.000000
2017-10-31,NaN,NaN


In [14]:
df3 = df2.dropna()

In [15]:
df3

,views,reads
published_date,,
2017-06-30,463.666667,112.333333
2017-07-31,5521.333333,1207.166667
2017-08-31,6242.800000,993.700000
2017-09-30,2113.000000,279.000000
2017-12-31,60818.750000,15724.000000
2018-01-31,31615.944444,9065.722222
2018-02-28,25463.875000,9933.750000
2018-03-31,69373.500000,18884.000000
2018-04-30,35668.333333,11340.333333


In [16]:
df3.iplot(
    kind='bar',
    xTitle='Дата',
    yTitle='Среднее',
    title='Среднемесячное кол-во просмотров и прочтений')

In [17]:
df2 = df[['views', 'reads',
          'published_date']].set_index('published_date').resample('M').sum()
df3=df2.dropna()

In [18]:
df3['read_percent']= df3['reads']/df3['views']
df3 = df3.dropna()

In [19]:
df3['read_percent'].iplot(
    kind='bar',
    xTitle='Дата',
    yTitle='Среднее',
    title='Доля дочитываний')

Кроме того можно на графике добавить еще одну ось Y

In [20]:
df2 = df[['views', 'read_time',
          'published_date']].set_index('published_date').resample('M').mean()

df2.iplot(
    kind='bar',
    xTitle='Дата',
    secondary_y='read_time',
    opacity=0.4,
    secondary_y_title='Среднее время прочтения',
    yTitle='Среднее кол-во просмотров',
    title='Средние за месяц')

Конечно же Plotly умеет рисовать и ящики с усами - куда же без них.

In [21]:
df[['claps', 'fans']].iplot(secondary_y='fans', secondary_y_title='Fans',
    kind='box', yTitle='Claps', title='BoxPlot Claps и Fans')

Можно нарисовать набор ящиков с усами, например, для нескольких категорий.

In [22]:
df2 = df.pivot(
    columns='publication', values='fans')
df2.head()

publication,Engineering @ Feature Labs,None,Noteworthy - The Journal Blog,The Reality Project,Towards Data Science
0,NaN,NaN,NaN,34.0,NaN
1,NaN,NaN,NaN,29.0,NaN
2,NaN,NaN,NaN,13.0,NaN
3,NaN,34.0,NaN,NaN,NaN
4,NaN,47.0,NaN,NaN,NaN


Всегда в iplot можно явно передать набор параметров в виде словаря(так у нас больше контроля)

In [23]:
df2.iplot(
        kind='box',
        layout=dict(
            height=600,
            yaxis=dict(title='fans'),
            title='Fans by Publication',
            margin=dict(b=140)))

In [24]:
df[df['read_time'] <= 10].pivot(
    columns='read_time', values='reads').iplot(
        kind='box', colorscale='set2',
        xTitle='Время прочтения',
        yTitle='Кол-во прочтений',
        title='Box Plot of Reads by Reading Time')

Сделаем из куска нашего датафрейма временной ряд. Для этого установим дату в индекс

In [25]:
tds = df[df['publication'] == 'Towards Data Science'].set_index(
    'published_date')

tds.head()

,claps,days_since_publication,fans,link,num_responses,publication,read_ratio,read_time,reads,started_date,...,word_count,claps_per_word,editing_days,<tag>Education,<tag>Data Science,<tag>Towards Data Science,<tag>Machine Learning,<tag>Python,time_started,time_published
published_date,,,,,,,,,,,,,,,,,,,,,
2017-12-27 11:20:00,4800,374.986885,861,https://towardsdatascience.com/random-forest-i...,27,Towards Data Science,17.68,21,28566,2017-12-26 15:11:00,...,4494,1.068091,0,0,1,0,1,1,15.183333,11.333333
2018-01-06 20:15:00,857,364.615092,112,https://towardsdatascience.com/improving-rando...,6,Towards Data Science,22.76,17,7207,2018-01-03 21:38:00,...,3504,0.244578,2,0,1,0,1,1,21.633333,20.250000
2018-01-07 20:37:00,186,363.599979,45,https://towardsdatascience.com/data-science-a-...,1,Towards Data Science,28.64,15,775,2018-01-07 13:18:00,...,3569,0.052115,0,0,1,0,0,0,13.300000,20.616667
2018-01-08 16:58:00,119,362.752029,43,https://towardsdatascience.com/a-theory-of-pre...,2,Towards Data Science,31.53,11,740,2018-01-02 17:23:00,...,2817,0.042244,5,0,1,0,0,0,17.383333,16.966667
2018-01-09 21:49:00,2000,361.550093,392,https://towardsdatascience.com/hyperparameter-...,12,Towards Data Science,23.99,12,25505,2018-01-09 12:26:00,...,2456,0.814332,0,0,1,0,1,1,12.433333,21.816667


In [26]:
tds['read_time'].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Дата',
    yTitle='Минимальное время прочтения',
    title='Read Time Trends')

Конечно же на одном графике можно отображать несколько рядов:

In [27]:
tds[['claps', 'fans']].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Fans and Claps',
    title='Fans and Claps over Time')

А можно явным образом настроить две оси Y.

In [28]:
tds[['fans', 'word_count', 'title']].iplot(
    y='fans',
    mode='lines+markers',
    secondary_y = 'word_count',
    secondary_y_title='Word Count',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Claps',
    text='title',
    title='Fans and Word Count over Time')